In [123]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pickle
import cv2
import time
from features import extract_features, calc_bin_spatial_features
%matplotlib inline

In [81]:
dist_pickle = pickle.load( open('Car_NoCar_LinearSVC.p', 'r' ) )
svc = dist_pickle["svc"]
X_scaler = dist_pickle["scaler"]
orient = dist_pickle["orientations"]
pix_per_cell = dist_pickle["pix_per_cell"]
cell_per_block = dist_pickle["cell_per_block"]
spatial_size = dist_pickle["color_spatial_size"]
h_colorspace = dist_pickle["h_colorspace"]
s_colorspace = dist_pickle["s_colorspace"]
#hog_channel = dist_pickle["hog_channel"]
hog_channel = 'ALL'

In [163]:
# Define a single function that can extract features using hog sub-sampling and make predictions
def find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, 
              spatial_size, cells_per_step, hog_channel='ALL', show_windows=True):
    
    draw_img = np.copy(img)
    img = img.astype(np.float32)/255
    
    img_roi = img[ystart:ystop,:,:]
    #ctrans_tosearch = convert_color(img_tosearch, conv='RGB2YCrCb')
    if scale != 1:
        imshape = img_roi.shape
        img_roi = cv2.resize(img_roi, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
    x_size = img_roi.shape[1]
    y_size = img_roi.shape[0]
    
    # Define blocks and steps as above
    #Number of Cells
    nxcells = x_size // pix_per_cell
    nycells = y_size // pix_per_cell
    #Number of Blocks
    nxblocks = (nxcells - cell_per_block) + 1 ##(W-F)/S + 1, W=ncells, F=cell_per_block, S=Stride=1
    nyblocks = (nycells - cell_per_block) + 1 
    #nxblocks = (x_size // pix_per_cell)-1  
    #nyblocks = (y_size // pix_per_cell)-1 
    nfeat_per_block = orient * (cell_per_block**2)
    
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = ((window // pix_per_cell) - cell_per_block) + 1
    
    #We have 8 cells per 64 pixels
    #Each cell step (8 pixels), refers to a move of 1/8 or overlap of 7/8=87.5% overlap
    #A step of 2 cells, means we slide windows by 16 pixels, overlap of 75%.
    #cells_per_step = 2  # Instead of overlap, define how many cells to step
    #Number of steps we can take over HoG array
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step
    
    # Compute individual channel HOG features for the entire image
    hog_array = extract_features(img_roi, h_cspace=h_colorspace, s_cspace=s_colorspace, orient=orient, 
                           pix_per_cell=pix_per_cell, cell_per_block=cell_per_block,
                           hog_channel=hog_channel, feature_vec=False, vis=False)
    #The hog_array we get has a shape 3XnyblocksXnxblocksXcell_per_blockXcell_per_blockXorientations
    
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            # Extract HOG for this patch
            hog_features = hog_array[:, ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            
            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell
            # Extract the image patch
            subimg = cv2.resize(img_roi[ytop:ytop+window, xleft:xleft+window, :], (64,64))
            
            # Get color features
            spatial_features = calc_bin_spatial_features(subimg, s_colorspace)
            #hist_features = color_hist(subimg, nbins=hist_bins)
            
            if show_windows:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                color = np.random.randint(0, 256, (3,))
                cv2.rectangle(draw_img,(xbox_left, ytop_draw+ystart), 
                              (xbox_left+win_draw, ytop_draw+win_draw+ystart),color,6) 
            # Scale features and make a prediction
            test_features = X_scaler.transform(np.hstack((hog_features, spatial_features)).reshape(1, -1))
            #test_features = X_scaler.transform(hog_features.reshape(1,-1))
            #test_features = X_scaler.transform(np.hstack((shape_feat, hist_feat)).reshape(1, -1))    
            test_prediction = svc.predict(test_features)
            
            if test_prediction == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                #color = np.random.randint(0, 256, (3,))
                cv2.rectangle(draw_img,(xbox_left, ytop_draw+ystart),
                              (xbox_left+win_draw, ytop_draw+win_draw+ystart),(0,0,255),6) 
             
    return draw_img

In [164]:
img = cv2.imread('test_images/test8.jpg') 
print(img.shape)

(738L, 1280L, 3L)


In [175]:
def find_cars_1(img):
    window = 64
    ystart = 400
    ystop = ystart + (window*3)
    scale = 1 #Steps 5
    t1 = time.time()
    out_img = find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell,
                        cell_per_block, spatial_size, cells_per_step=5, hog_channel=hog_channel, show_windows=False)
    return out_img
#print("Time taken", time.time()-t1)
#plt.imshow(cv2.cvtColor(out_img, cv2.COLOR_BGR2RGB))

In [176]:
def find_cars_2(img):
    ystart = 400 + (window)
    ystop = ystart + (window*6)
    scale = 1.8 #steps 4

    t1 = time.time()
    out_img = find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell,
                        cell_per_block, spatial_size, cells_per_step=6, hog_channel=hog_channel, show_windows=False)
    #print("Time taken", time.time()-t1)
    return out_img
#plt.imshow(cv2.cvtColor(out_img, cv2.COLOR_BGR2RGB))
#plt.show()

In [171]:
def combine(img1, img2):
    return (cv2.addWeighted(img1, 0.5, img2, 0.5, 0))

In [187]:
cap = cv2.VideoCapture('project_video.mp4')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
recorder = cv2.VideoWriter('out.mp4', fourcc, 30, (1280, 720))
t1 = time.time()
cnt = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        cnt+=1;
        #print("cnt", cnt)
        if cnt == 300:
            break
        if cnt%10 == 0:
            print('Time taken ', time.time()-t1)
        f1 = find_cars_1(frame)
        f2 = find_cars_2(frame)
        #print(f1.shape)
        #print(f2.shape)
        img_boxes = combine(find_cars_1(frame), find_cars_2(frame))
        recorder.write(img_boxes)
    else:
        break
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
print('Time taken ', time.time()-t1)
cap.release()
recorder.release()
cv2.destroyAllWindows()

('Time taken ', 8.181999921798706)
('Time taken ', 17.19099998474121)
('Time taken ', 25.91599988937378)
('Time taken ', 34.86199998855591)
('Time taken ', 43.64199995994568)
('Time taken ', 52.36500000953674)
('Time taken ', 60.88199996948242)
('Time taken ', 69.63800001144409)
('Time taken ', 78.32999992370605)
('Time taken ', 87.35199999809265)
('Time taken ', 98.34299993515015)
('Time taken ', 108.32200002670288)
('Time taken ', 116.92499995231628)
('Time taken ', 125.66599988937378)
('Time taken ', 134.3769998550415)
('Time taken ', 143.1729998588562)
('Time taken ', 152.06299996376038)
('Time taken ', 160.74399995803833)
('Time taken ', 169.41100001335144)
('Time taken ', 178.1289999485016)
('Time taken ', 187.15199995040894)
('Time taken ', 195.83899998664856)
('Time taken ', 204.41100001335144)
('Time taken ', 213.07800006866455)
('Time taken ', 222.07500004768372)
('Time taken ', 230.63299989700317)
('Time taken ', 239.50799989700317)
('Time taken ', 248.3550000190735)
('Time 